In [ ]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData
from sqlalchemy import Table, select, func, and_, insert, delete, update, or_

In [ ]:
import json

sql_credentials = json.load(open('/root/thomas/sqlcredentials.json'))

In [ ]:
from sqlalchemy import create_engine

sql_engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                                         sql_credentials["host"], sql_credentials["port"],
                                         sql_credentials["database"]))


Lati stuff

In [ ]:
metadata = MetaData()
# step 1 - download crops + json
fish_crops = Table('lati_fish_detections', metadata, autoload=True, autoload_with=sql_engine)
lice_crops = Table('lati_fish_detections_lice_annotations', metadata, autoload=True,
                   autoload_with=sql_engine)

# inner join on fish crop id
query = select([fish_crops.c.site_id,
                fish_crops.c.pen_number,
                fish_crops.c.metadata,
                lice_crops.c.is_skipped,
                lice_crops.c.is_blurry,
                lice_crops.c.is_too_dark,
                lice_crops.c.is_obstructed,
                lice_crops.c.is_bad_crop,
                fish_crops.c.created_at
                ]) \
    .select_from(lice_crops.join(fish_crops, lice_crops.c.lati_fish_detections_id == fish_crops.c.id)) \
    .where(fish_crops.c.created_at >= '2019-04-01')

In [ ]:
connection = sql_engine.connect()
q = connection.execute(query)
results = [r for r in q]

In [ ]:
import pandas as pd

In [ ]:
dataset = {'site_id':[],
           'pen_number': [],
           'metadata': [],
           'blurriness': [],
           'darkness': [],
           'quality': [],
           'is_skipped': [],
           'is_blurry': [],
           'is_too_dark': [],
           'is_obstructed': [],
           'is_bad_crop': [],
           'created_at': []
           }

In [ ]:
for r in results:
    metadata = r[2]
    if 'qualityScore' not in metadata:
        continue
    dataset['site_id'].append(r[0])
    dataset['pen_number'].append(r[1])
    
    dataset['metadata'].append(metadata)
    dataset['blurriness'].append(metadata['qualityScore']['blurriness'])
    dataset['quality'].append(metadata['qualityScore']['quality'])
    dataset['darkness'].append(metadata['qualityScore']['darkness'])
    
    dataset['is_skipped'].append(r[3])
    dataset['is_blurry'].append(r[4])
    dataset['is_too_dark'].append(r[5])
    dataset['is_obstructed'].append(r[6])
    dataset['is_bad_crop'].append(r[7])
    dataset['created_at'].append(r[8])

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
df.to_csv('/root/data/rds/lati_april.csv')

Bati stuff

In [ ]:
metadata = MetaData()
# step 1 - download crops + json
crops = Table('fish_detections', metadata, autoload=True, autoload_with=sql_engine)
kps = Table('keypoint_annotations', metadata, autoload=True, autoload_with=sql_engine)

# inner join on fish crop id
query = select([crops.c.site_id,
                crops.c.pen_id,
                crops.c.left_crop_metadata,
                crops.c.right_crop_metadata,
                kps.c.is_skipped,
                kps.c.is_blurry,
                kps.c.is_dark,
                kps.c.is_occluded,
                crops.c.captured_at
                ]) \
    .select_from(kps.join(crops, kps.c.fish_detection_id == crops.c.id)) \
    .where(crops.c.captured_at >= '2019-04-01')

In [ ]:
connection = sql_engine.connect()
q = connection.execute(query)
results = [r for r in q]

In [ ]:
dataset = {'site_id':[],
           'pen_id': [],
           
           'left_metadata': [],
           'left_blurriness': [],
           'left_darkness': [],
           'left_quality': [],
           
           'right_metadata': [],
           'right_blurriness': [],
           'right_darkness': [],
           'right_quality': [],
           
           'is_skipped': [],
           'is_blurry': [],
           'is_dark': [],
           'is_occluded': [],
           'captured_at': []
           }

In [ ]:
for r in results:
    metadata = r[2]
    if 'qualityScore' not in metadata:
        continue
    dataset['site_id'].append(r[0])
    dataset['pen_id'].append(r[1])
    
    dataset['left_metadata'].append(metadata)
    dataset['left_blurriness'].append(metadata['qualityScore']['blurriness'])
    dataset['left_quality'].append(metadata['qualityScore']['quality'])
    dataset['left_darkness'].append(metadata['qualityScore']['darkness'])
    
    metadata = r[3]
    dataset['right_metadata'].append(metadata)
    dataset['right_blurriness'].append(metadata['qualityScore']['blurriness'])
    dataset['right_quality'].append(metadata['qualityScore']['quality'])
    dataset['right_darkness'].append(metadata['qualityScore']['darkness'])
    
    dataset['is_skipped'].append(r[4])
    dataset['is_blurry'].append(r[5])
    dataset['is_dark'].append(r[6])
    dataset['is_occluded'].append(r[7])
    dataset['captured_at'].append(r[8])

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.shape

In [ ]:
df.to_csv('/root/data/rds/bati_april.csv')